# *Importing Modules*
---

In [1]:

from tabula import read_pdf
#from tabulate import tabulate
#import camelot
import pandas as pd
import numpy as np
import re

# *Reading pdf file containing table*
---

In [2]:
input_file = read_pdf("C:/Users/Atal/PycharmProjects/MachineLearningImplementation/test_input.pdf",
                    pages=1,java_options="-Dfile.encoding=UTF8", stream=True)

In [3]:
input_file[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Value date   24 non-null     object
 1   Order date   19 non-null     object
 2   Description  30 non-null     object
 3   Amount       26 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB


In [4]:
table = pd.DataFrame(input_file[0])
table.head()

,Value date,Order date,Description,Amount
0,Deposits,NaN,NaN,NaN
1,Deposits AUD,NaN,Opening balance,32.29
2,NaN,NaN,Month end balance(31 Mar 2019),32.29
3,Deposits HKD,NaN,Opening balance,0.00
4,01 Mar 2019,28 Feb 2019,INTEREST RECEIVED,2.39


---
## *Pattern matching to get the Currency type*

### *['AUD', 'HKD', 'SGD', 'USD']*

In [5]:
pattern_currency = re.compile('[A-Z]{3}')

In [6]:
Currency = []
# empty list to hold currency type

for i in range(len(table)):
    match = pattern_currency.findall(str(table.loc[i, 'Value date']))
    if i == 0:
        Currency.append(np.nan)
    elif match:
        Currency.append(match)
    else:
        Currency.append(Currency[i-1])

In [7]:
# Converting the list of list into a list of string

for i in range(1,len(Currency)):
    Currency[i] = Currency[i][0]

In [8]:
# Inserting Currency column to the dataframe table

table['Currency'] = Currency
table.head()

,Value date,Order date,Description,Amount,Currency
0,Deposits,NaN,NaN,NaN,NaN
1,Deposits AUD,NaN,Opening balance,32.29,AUD
2,NaN,NaN,Month end balance(31 Mar 2019),32.29,AUD
3,Deposits HKD,NaN,Opening balance,0.00,HKD
4,01 Mar 2019,28 Feb 2019,INTEREST RECEIVED,2.39,HKD


## *Pattern matching for transaction type, i.e.*
+ ### *Credit or*
+ ### *Debit*

In [9]:
tran_patt = re.compile('[(][\d(),.]+[)]')
Credit = [np.nan]*len(table)
Debit = [np.nan]*len(table)

In [10]:
for i in range(len(table)):
    match_tran = tran_patt.findall(str(table.loc[i,'Amount']))
    if match_tran:
        Debit[i] = match_tran
    else:
        Credit[i] = table.loc[i,'Amount']

In [11]:
print("Debit",Debit)
print("\n")
print("Credit",Credit)

Debit [nan, nan, nan, nan, nan, nan, nan, nan, nan, ['(5,443,428.95)'], ['(1,834.73)'], nan, ['(5,445,263.68)'], ['(1,998.34)'], nan, ['(2,427,234.63)'], ['(849.20)'], nan, nan, nan, nan, nan, ['(2,428,083.83)'], ['(881.01)'], nan, nan, nan, nan, nan, nan, nan]


Credit [nan, '32.29', '32.29', '0.00', '2.39', '20,025.00', nan, '3,000,000.00', nan, nan, nan, '5,445,263.68', nan, nan, '2,427,234.63', nan, nan, '2,428,083.83', '20,025.00', nan, '3,000,000.00', nan, nan, nan, '591,060.16', '302,404.96', '302,404.96', '64,338.48', '15.50', '1,325.50', '400.00']


In [12]:
# Debit is list of list so converting it into list of floating numbers

for i in range(len(Debit)):
    temp_y = []
    if not pd.isnull(Debit[i]):
        for j in Debit[i][0]:
            if j != "'" and j != "(" and j != ")" and j != ",":
                temp_y.append(j)
        temp_y = "".join(temp_y)
        Debit[i] = float(temp_y)

In [13]:
# Credit is list of list so converting it into list of floating numbers

for i in range(len(Credit)):
    y = []
    if not pd.isnull(Credit[i]):
        for j in Credit[i]:
            if j != "'" and j != "(" and j != ")" and j != ",":
                y.append(j)
        y = "".join(y)
        Credit[i] = float(y)

In [14]:
# Inserting columns Debit and Credit as 4th and 5th column of the dataframe respectively

table.insert(3,'Debit',Debit)
table.insert(4,'Credit',Credit)

In [15]:
table.head()

,Value date,Order date,Description,Debit,Credit,Amount,Currency
0,Deposits,NaN,NaN,NaN,NaN,NaN,NaN
1,Deposits AUD,NaN,Opening balance,NaN,32.29,32.29,AUD
2,NaN,NaN,Month end balance(31 Mar 2019),NaN,32.29,32.29,AUD
3,Deposits HKD,NaN,Opening balance,NaN,0.00,0.00,HKD
4,01 Mar 2019,28 Feb 2019,INTEREST RECEIVED,NaN,2.39,2.39,HKD


## *Removing the section headers*

In [16]:
# Collecting the indexes of rows formed by section headers

sec_header1 = re.compile('^[D][\w\s]+')
sec_header2 = re.compile('^[M|O][\w\s]+')
# Pattern matching for section header

idx = []
for i in range(len(table)):
    temp_header1 = sec_header1.findall(str(table.loc[i,'Value date']))
    temp_header2 = sec_header2.findall(str(table.loc[i,'Description']))
    if temp_header1 or temp_header2:
        idx.append(i)
idx # Collection of row index for section headers

[0, 1, 2, 3, 24, 25, 26, 27]

In [17]:
# Dropping section header rows

table = table.drop(idx)

In [18]:
table.head()

,Value date,Order date,Description,Debit,Credit,Amount,Currency
4,01 Mar 2019,28 Feb 2019,INTEREST RECEIVED,NaN,2.39,2.39,HKD
5,04 Mar 2019,27 Feb 2019,INTEREST FOR BNP PARIBAS 4M CALLABLE FIXED COU...,NaN,20025.00,"20,025.00",HKD
6,NaN,NaN,ISIN:XS1881628199 #44368-0,NaN,NaN,NaN,HKD
7,04 Mar 2019,04 Mar 2019,"EARLY REDEMPTION OF HKD 3,000,000.00 BNP PARIB...",NaN,3000000.00,"3,000,000.00",HKD
8,NaN,NaN,04MAR2019 8.01% ISIN:XS1881628199 #12740-0,NaN,NaN,NaN,HKD


## *Formatting the date values in column 'Value date' and 'Order Date'*
---

In [19]:
table['Value date'], table['Order date'] = pd.to_datetime(table['Value date']), pd.to_datetime(table['Order date'])

In [20]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 4 to 30
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Value date   19 non-null     datetime64[ns]
 1   Order date   19 non-null     datetime64[ns]
 2   Description  23 non-null     object        
 3   Debit        8 non-null      float64       
 4   Credit       11 non-null     float64       
 5   Amount       19 non-null     object        
 6   Currency     23 non-null     object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 1.4+ KB


For some transactions the trans action descriptions got split into two separate rows.
Here concatenating the descriptions for the same transaction, and then dropping the description row.

In [21]:
for i in table.index:
    if pd.isnull(table['Value date'])[i]:
        j = table.loc[i-1,'Description']+" "+table.loc[i,'Description']
        table = table.replace(to_replace = [table.loc[i-1,'Description']],value = j)
        table = table.drop(i)

In [22]:
# Dropping the 'Amount' column

table.drop('Amount',inplace = True,axis = 1)

In [23]:
# Replacing the NaN cell values with blank("")

table = table.fillna("")
table.head()

,Value date,Order date,Description,Debit,Credit,Currency
4,2019-03-01,2019-02-28,INTEREST RECEIVED,,2.39,HKD
5,2019-03-04,2019-02-27,INTEREST FOR BNP PARIBAS 4M CALLABLE FIXED COU...,,20025.0,HKD
7,2019-03-04,2019-03-04,"EARLY REDEMPTION OF HKD 3,000,000.00 BNP PARIB...",,3000000.0,HKD
9,2019-03-05,2019-03-05,REPAY FIXED LOAN #31806-0,5443428.95,,HKD
10,2019-03-05,2019-03-05,INTEREST CHARGE ON FIXED LOAN #31806-0,1834.73,,HKD


In [24]:
# Saving the dataframe as Excel(.xlsx) file
table.to_excel("Output_table.xlsx",index = None)
